In [1]:
import numpy as np

In [4]:
# reading and processing text
with open('dataset/1268-0.txt') as fp:
    text = fp.read()

start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')

text = text[start_indx:end_indx]

char_set = set(text)

print('Total length: ', len(text))
print('Unique characters: ', len(char_set))

Total length:  1130712
Unique characters:  85


In [6]:
chars_sorted = sorted(char_set)

char2int = {ch: i for i, ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)

In [8]:
text_encoded = np.array([char2int[ch] for ch in text], dtype=np.int32)

print('Text encoded shape: ', text_encoded.shape)
print(text[:15], '== Encoding ==>', text_encoded[:15])

print(text_encoded[15:21], '== Reverse ==>',''.join(char_array[text_encoded[15:21]]))

Text encoded shape:  (1130712,)
THE MYSTERIOUS  == Encoding ==> [48 36 33  1 41 53 47 48 33 46 37 43 49 47  1]
[37 47 40 29 42 32] == Reverse ==> ISLAND


In [9]:
import tensorflow as tf

2022-12-17 18:32:03.771258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-17 18:32:04.193002: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-17 18:32:04.193040: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-17 18:32:05.385731: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [10]:
ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))

48 -> T
36 -> H
33 -> E
1 ->  
41 -> M


2022-12-17 18:33:16.544966: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-17 18:33:16.545053: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: harsh-IdeaPad-Gaming3-15ARH05D
2022-12-17 18:33:16.545067: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: harsh-IdeaPad-Gaming3-15ARH05D
2022-12-17 18:33:16.545245: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.65.1
2022-12-17 18:33:16.545657: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  515.65.01  Release Build  (dvs-builder@U16-T11-05-2)  Wed Jul 20 13:54:56 UTC 2022
GCC version:

In [11]:
seq_length = 40
chunk_size = seq_length + 1

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)

# define the function for splitiing x and y

def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

In [12]:
for example in ds_sequences.take(2):
    print('Input (X) : ', repr(''.join(char_array[example[0].numpy()])))
    print('Target (y): ', repr(''.join(char_array[example[1].numpy()])))
    print()

Input (X) :  'THE MYSTERIOUS ISLAND ***\n\n\n\n\nTHE MYSTER'
Target (y):  'HE MYSTERIOUS ISLAND ***\n\n\n\n\nTHE MYSTERI'

Input (X) :  'OUS ISLAND\n\nby Jules Verne\n\n1874\n\n\n\n\nPAR'
Target (y):  'US ISLAND\n\nby Jules Verne\n\n1874\n\n\n\n\nPART'



In [13]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

Building a character-level **RNN Mdoel**

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(
            rnn_units,
            return_sequences=True
        ),
        tf.keras.layers.Dense(vocab_size)
    ])

    return model

In [15]:
# setting the trainig parameters
charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(1)
model = build_model(
    vocab_size=charset_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         21760     
                                                                 
 lstm (LSTM)                 (None, None, 512)         1574912   
                                                                 
 dense (Dense)               (None, None, 85)          43605     
                                                                 
Total params: 1,640,277
Trainable params: 1,640,277
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True
    )
)

model.fit(ds, epochs=20)

Epoch 1/20
431/431 [==============================] - 177s 403ms/step - loss: 2.3260
Epoch 2/20
431/431 [==============================] - 149s 343ms/step - loss: 1.7309
Epoch 3/20
431/431 [==============================] - 149s 343ms/step - loss: 1.5269
Epoch 4/20
431/431 [==============================] - 197s 455ms/step - loss: 1.4139
Epoch 5/20
431/431 [==============================] - 216s 497ms/step - loss: 1.3428
Epoch 6/20
431/431 [==============================] - 209s 481ms/step - loss: 1.2935
Epoch 7/20
431/431 [==============================] - 187s 431ms/step - loss: 1.2557
Epoch 8/20
431/431 [==============================] - 174s 400ms/step - loss: 1.2259
Epoch 9/20
431/431 [==============================] - 195s 449ms/step - loss: 1.2006
Epoch 10/20
431/431 [==============================] - 217s 501ms/step - loss: 1.1783
Epoch 11/20
431/431 [==============================] - 195s 448ms/step - loss: 1.1589
Epoch 12/20
431/431 [==============================] - 213s 490

In [17]:
# an example of auto regression

def sample(model, starting_str, len_generated_text=500, max_input_length=40, scale_factor=1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1, -1))

    generated_str = starting_str

    model.reset_states()

    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)
        # to remove unwanted dimensions

        scaled_logits = logits * scale_factor

        new_char_indx = tf.random.categorical(scaled_logits, num_samples=1)
        new_char_indx = tf.squeeze(new_char_indx)[-1].numpy()

        generated_str += str(char_array[new_char_indx])

        new_char_indx = tf.expand_dims([new_char_indx], 0)
        # adds a new dimension

        encoded_input = tf.concat([encoded_input, new_char_indx], axis=1)
        encoded_input = encoded_input[:, -max_input_length:]
    return generated_str

In [19]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island'))

The island may have been exheard. But these woods effected into the corral. Top and if
we are going to lear out necessary to stop her to be convenient
for the wound!

In fact, Pencroft, to keeping the house, and still strenched to the cliff which would ran place some emotion, the point was secret on the islet besides after, and
Herbert, under all they publigged but to set, to then be
mentioned; Captain Harding,” replied Cyrus Harding. “The colonists, were falled one of the colonists; the wind, and
this qu


In [20]:
print(sample(model, starting_str='The island', scale_factor=2.0))

The island was being worked them
in the corral. The colonists were confided to the engineer and his companions were not attacked on the shore.

The colonists were not reason to prevent it.”

“And what was the box was a large number, a convicts for the corral alone to the attacks of the colonists like the island.

The colonists were determined to the colonists were to be feared that the stores of lava would have
been easily as a sort of all that the others worked with the corral.

Herbert and Herbert, and 
